#### <u>Sudoku Solver (CSC-462)</u> 

In [1]:
from pprint import pprint
from math import inf

""" Problem Representation """
sudoku = [[0 for col in range(9)] for row in range(9)]

<u>Constraints Satisfier</u>

In [2]:
""" Check Constraints """
def check_constraints(r, c, value):
    #* input: `r` representing the row and `c` representing the column 
    #* in which the provided `value` is to be inserted,
    #* output: bool:- True||False depending on whether the constraints are met by the given value or not
    
    # 1st - Check if any variable in the col of the specified cell has the same value
    for y in range(9): #? O(9) unless there is a change in the sudoku architecture
        if sudoku[r][y] == value: #? Basic Operation : Comparision(==)
            return False

    # 2nd - Check if any variable in the row the specified cell has the same value
    for x in range(9): #? O(9) unless there is a change in the sudoku architecture
        if sudoku[x][c] == value: #? Basic Operation : Comparision(==)
            return False
    
    
    # 3rd - Check if thers is any variable with the same value within the specific subgrid
    
    # Figuring out the subgrid that the CELL(r,c) belongs to
    col_low = 3 * (c//3)
    col_up = col_low + 2

    row_low = 3 * (r//3)
    row_up = row_low + 2

    # Checking if we have any same element in the subgrid
    for x in range(row_low, row_up+1):
        for y in range(col_low, col_up+1):

            if (x,y) != (r,c) and sudoku[x][y] == value:
                return False



    return True


<u>Problem Generator</u>

In [3]:
import random

""" Problem Generator """
def problem_generator(l, h):
    # given lower and upper limit for how much variables need to be assigned 
    # before starting to solve the problem

    fixed = random.randint(l, h)
    
    for i in range(fixed):
        r = random.randint(0,8)
        c = random.randint(0,8)

        value = random.randint(1,9)
        while(not check_constraints(r,c,value) or sudoku[r][c] != 0):

            if (sudoku[r][c]!=0):
                r = random.randint(0,8)
                c = random.randint(0,8)
            else:
                value = random.randint(1,9)
                

        sudoku[r][c] = value


problem_generator(12,18)
pprint(sudoku)

[[0, 0, 0, 0, 6, 0, 0, 9, 0],
 [0, 3, 6, 8, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 8, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 7, 8, 0],
 [0, 0, 7, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 4, 0],
 [0, 0, 0, 5, 2, 1, 0, 0, 0],
 [4, 0, 1, 0, 9, 6, 0, 0, 0]]


---
Now that we have the Constraint Checker, we can start using the Backtracking Search in order to solve the problem.

Starting by mantaining a record of Legal Domain Values.

In [4]:
# Legal Domain

""" A map of each variable's legal domain 
containing key: value pairs
where key represents the Cell
and value is the set of available domain values
"""
legal_domain = {}


def init_domain():

    for x in range(9):
        for y in range(9):

            if sudoku[x][y] == 0:
                legal_domain[(x,y)] = [x for x in range(1,10)]
            else:
                legal_domain[(x,y)] = None

def recalculate_domain(r, c):

    # Recalculate the Domain
    if legal_domain.get((r,c)) != None:
        for v in range(1,10):
            if (check_constraints(r,c,v)) and v not in legal_domain.get((r,c)):
                legal_domain.get((r,c)).append(v)


def expand_domain(r, c):
    # Rewriting the update constraints 
    
    # 1. Reason for change: 
    # Operating Backtracking Recursively. Hence, if one failure occurs we 
    # can backtrack and use any other value assignment in hope that it may
    # lead us to a solution

    # 2. Flaw in previous approach:
    # Not mantaining the table on every recursive call and if a failure happens,
    # we were not able to revert back to the previous constraints.

    # New Approach: This time the table will update on every new assignment and
    # is also able to revert back if the assignment if reverted

    # Update constraints by recalculating the domain of every variable in the territory

    # 1. Row variables
    # for every row variable
    for y in range(9):

        # Recalculate the Domain
        recalculate_domain(r,y)

    # 2. Column Variables
    # for every column Variable
    for x in range(9):

        # Recalculate the Domain
        recalculate_domain(x, c)
        
    # 3. for the subgrid variables
    # for every variable in the subgrid

    c_low = 3 * (c//3)
    c_up = c_low + 2

    r_low = 3 * (r//3)
    r_up = r_low + 2
    
    for x in range(r_low, r_up+1):
        for y in range(c_low, c_up+1):
            
            # Recalculate the Domain
            recalculate_domain(x,y)

def reduce_domain(r, c, value):

    """ We will do this in three steps
    i). Reduce legal Domain of concerned variables in row
    ii). Reduce legal Domain of concerned variables in column
    iii). Reduce legal Domain of concerned variables in subgrid
    """

    #! i).
    for y in range(0,9):
        
        # if the variable is assigned; continue
        if legal_domain.get((r,y)) == None:
            continue;

        for v in legal_domain.get((r,y)):
            if v == value:
                legal_domain.get((r,y)).remove(v)


    #! ii).
    for x in range(0,9):
        
        # if the variable is assigned; continue
        if legal_domain.get((x,c)) == None:
            continue;

        # if it's not
        for v in legal_domain.get((x,c)):
            if v == value:
                legal_domain.get((x,c)).remove(v)

    #! iii).
    rl = 3 * (r//3)
    ru = rl + 2

    cl = 3 * (c//3)
    cu = cl + 2

    # print(f"rl: {rl}, ru: {ru}, cl: {cl}, cu: {cu}")

    for x in range(rl, ru+1):
        for y in range(cl, cu+1):
        

            # if the variable is assigned; continue
            if legal_domain.get((x,y)) == None:
                continue;

            for v in legal_domain.get((x,y)):
                if v == value:
                    legal_domain.get((x,y)).remove(v)

            
# initializing the domain
init_domain()

# intializing the constraints
for x in range(9):
    for y in range(9):
        reduce_domain(x,y,sudoku[x][y])

pprint(legal_domain)

{(0, 0): [1, 2, 5, 7, 8],
 (0, 1): [1, 2, 4, 5, 7, 8],
 (0, 2): [2, 4, 5],
 (0, 3): [2, 3, 4, 7],
 (0, 4): None,
 (0, 5): [2, 3, 4, 5, 7],
 (0, 6): [1, 2, 3, 4, 5, 8],
 (0, 7): None,
 (0, 8): [1, 2, 3, 4, 5, 7, 8],
 (1, 0): [1, 2, 5, 7, 9],
 (1, 1): None,
 (1, 2): None,
 (1, 3): None,
 (1, 4): [1, 4, 5, 7],
 (1, 5): [2, 4, 5, 7, 9],
 (1, 6): [1, 2, 4, 5],
 (1, 7): [1, 2, 5, 7],
 (1, 8): [1, 2, 4, 5, 7],
 (2, 0): [1, 2, 5, 7, 8, 9],
 (2, 1): [1, 2, 4, 5, 7, 8, 9],
 (2, 2): [2, 4, 5, 9],
 (2, 3): [2, 3, 4, 7, 9],
 (2, 4): [1, 3, 4, 5, 7],
 (2, 5): [2, 3, 4, 5, 7, 9],
 (2, 6): [1, 2, 3, 4, 5, 6, 8],
 (2, 7): [1, 2, 3, 5, 6, 7],
 (2, 8): [1, 2, 3, 4, 5, 6, 7, 8],
 (3, 0): [1, 2, 3, 5, 6, 9],
 (3, 1): [1, 2, 4, 5, 6, 9],
 (3, 2): None,
 (3, 3): [2, 3, 4, 6, 7, 9],
 (3, 4): [3, 4, 5, 7],
 (3, 5): [2, 3, 4, 5, 7, 9],
 (3, 6): [1, 2, 3, 4, 5, 6, 9],
 (3, 7): [1, 2, 3, 5, 6],
 (3, 8): [1, 2, 3, 4, 5, 6, 9],
 (4, 0): [1, 2, 3, 5, 6, 9],
 (4, 1): [1, 2, 4, 5, 6, 9],
 (4, 2): [2, 3, 4, 5, 9],
 (4,

---
Having set up the utilites to keep the legal_domain for each variable updated
We can move on <u>`Backtracking Search`</u>

In [5]:
# def select_unassigned_variable(solution):

#     for i in range(9):
#         for j in range(9):
#             if solution[i][j] == 0:
#                 return (i, j)


def is_complete(solution):
    for i in range(9):
        for j in range(9):
            if solution[i][j] == 0:
                return False

    return True

def unassigned_variables():
    uav = []

    for x in range(9):
        for y in range(9):
            if sudoku[x][y] == 0:
                uav.append((x,y))
    
    return uav

def mrv():
    
    # mdv - min_domain_value
    # multiple minimum domain variables - incase if we have more than one
    mdv = inf
    mmdv = []
    
    # getting the unassigned variables
    uav = unassigned_variables()

    # check the domains of the variables to get the smallest domain
    for var in uav:
        if len(legal_domain.get(var)) < mdv:
            mdv = len(legal_domain.get(var))
            mmdv.clear()
            mmdv.append(var)
        elif len(legal_domain.get(var)) == mdv:
            mmdv.append(var)


    return mmdv



def constrains(var):

    # Steps to solve this
    # 1. Check how many unassigned variable are in same row
    # 2. Check how many unassigned variable are in same col
    # 3. Check how many unassigned variable are in same sub-grid
    impact_degree = 0

    # 1.
    for y in range(9):
        if sudoku[var[0]][y] == 0:
            impact_degree += 1

    # 2. 
    for x in range(9):
        if sudoku[x][var[1]] == 0:
            impact_degree += 1

    # 3.
    r_low = 3 * (var[0] // 3)
    r_high = r_low + 2

    c_low = 3 * (var[1] // 3)
    c_high = c_low + 2

    for x in range(r_low, r_high+1):
        for y in range(c_low, c_high+1):
            if sudoku[x][y] == 0:
                impact_degree += 1

    return impact_degree;

def deg_heuristic(vars):

    # for each of the variables calculate the impact they have
    impact = {}

    # Steps to solve this problem:
    # 1. Check how many unassigned variables are in each variables territory
    # (that number will display the impact of that variable assignment on other variables)
    # 2. pick the one that has maximum

    # 1.
    for var in vars:
        impact.update({var: constrains(var)})
    
    # 2.
    max_impact = -inf
    max_impact_var = None 
    
    for var in impact:
        if impact.get(var) > max_impact:
            max_impact = impact.get(var)
            max_impact_var = var

    return max_impact_var

def least_constraining_value(var):

    lcv = None
    lcc = inf

    # print(var)
    # print(legal_domain.get(var))
    # if len(legal_domain.get(var)) == 0:
        # pprint(sudoku)
    # if len(legal_domain.get(var)) == 1:
    #     return legal_domain.get(var)[0];


    # Since the value is in the legal_domain it satisfies the constraints
    for value in legal_domain.get(var):
        
        # The value itself must satisfy the constraints
        # if (not check_constraints(var[0], var[1], value)):
        #     continue

        # We will calculate the constraint count for each value in the var domain
        constraint_count = 0

        # Since only the territory Variables can be affected
        # we will check in that
        # 
        # 1. Row
        # 2. Column
        # 3. SubGrid

        # 1.
        for y in range(9):
            if legal_domain.get((var[0], y)) != None and (var[0], y) != var:
                if value in legal_domain.get((var[0], y)):
                    constraint_count += 1

        # 2.
        for x in range(9):
            if legal_domain.get((x, var[1])) != None and (x, var[1]) != var:
                if value in legal_domain.get((x, var[1])):
                    constraint_count += 1

        # 3.
        rl = 3 * (var[0]//3)
        ru = rl + 2

        cl = 3 * (var[1]//3)
        cu = cl + 2

        for x in range(rl, ru+1):
            for y in range(cl, cu+1):
                if legal_domain.get((x,y))!=None and (x,y) != var:
                    if value in legal_domain.get((x,y)):
                        constraint_count += 1
        

        if lcc > constraint_count:
            lcc = constraint_count
            lcv = value

    # print(lcv, lcc)
    assert lcv != None
    return lcv    

def backtracking_search(solution):

    # Base Condition
    if is_complete(solution):
        return solution

    # TO-DOne  CHANGE (instead of randomly getting variable to assign, choose one with min domain)
    # var = select_unassigned_variable(solution)
    var = mrv()

    # check if there are more than one variables with the min domain
    # calculate the heuristic if there are and choose one with max impact
    if len(var) > 1:
        var = deg_heuristic(var)
    else:
        var = var[0]
    
    # TO-DOne Instead of all the values in the legal_domain of that variable, select the value
    # that has the least impact on the variables in it's territory
    for value in legal_domain[var]:
        # if check_constraints(var[0], var[1], value):
    # value = least_constraining_value(var)
    
        solution[var[0]][var[1]] = value
        reduce_domain(var[0], var[1], value)

        result = backtracking_search(solution)
        if result != None:
            return result

        solution[var[0]][var[1]] = 0
        expand_domain(var[0], var[1]);

    return None

In [6]:
""" Generated Problem """
# from time import time

# st = time()
backtracking_search(sudoku)
# et = time()

# print(f"Time taken: {round(et-st,5)}s")

In [7]:
""" Parsed Data """

from time import time

data = open("problems.csv")

data.readline()

st = time()
for line in data:
    problem = line.strip().split(",")[0]
    sudoku = [[0 for col in range(9)] for row in range(9)]
    
    for x in range(9):
        for y in range(9):
            sudoku[x][y] = int(problem[(9*x)+y])

            init_domain();

            # intializing the constraints
            for x in range(9):
                for y in range(9):

                    if sudoku[x][y] == 0:
                        continue

                    reduce_domain(x,y,sudoku[x][y])
    
    print("Problem: ")
    pprint(sudoku)
    print("Solution: ")
    pprint(backtracking_search(sudoku))

et = time()

# data.write(f"Time taken: {round(et-st,5)}s");
print(f"Time taken: {round(et-st,5)}s")

Problem: 
[[0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [6, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 4, 2, 9, 1, 0, 3, 0, 0]]
Solution: 
[[2, 3, 1, 7, 9, 5, 4, 8, 6],
 [4, 6, 9, 8, 2, 1, 7, 3, 5],
 [8, 7, 5, 6, 4, 3, 9, 1, 2],
 [7, 2, 4, 3, 8, 9, 6, 5, 1],
 [1, 8, 3, 4, 5, 6, 2, 9, 7],
 [9, 5, 6, 1, 7, 2, 8, 4, 3],
 [6, 1, 8, 2, 3, 4, 5, 7, 9],
 [3, 9, 7, 5, 6, 8, 1, 2, 4],
 [5, 4, 2, 9, 1, 7, 3, 6, 8]]
Problem: 
[[0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0],
 [5, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [8, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [6, 0, 0, 0, 0, 0, 0, 0, 0],
 [2, 5, 0, 0, 9, 7, 1, 0, 0]]
Solution: 
[[7, 3, 2, 8, 5, 4, 9, 6, 1],
 [1, 8, 6, 7, 2, 9, 4, 5, 3],
 [5, 4, 9, 3, 6, 1, 7, 8, 2],
 [3, 6, 1, 5, 7, 2, 8, 9, 4],
 [9, 2, 5, 4, 8, 6, 3, 1, 7]